In [1]:
from __future__ import print_function
import sys
import copy
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import cPickle as pickle
import cv2
import math3d as m3d
import versor as vsr
from motor_estimation import MotorEstimationSolver
from hand_eye_calibration import HandEyeCalibrator
from chessboard import calibrate_lens, find_corners, get_object_pose

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Object points

In [2]:
pattern_size = (10,6)
square_size = 0.02
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float64)
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1,2)
pattern_points *= square_size

### Robot poses

In [3]:
rob_pose_list = pickle.load(open('pose_list.dump'))
rob_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in rob_pose_list]]

### Object poses

In [4]:
obj_pose_list = pickle.load(open('obj_pose_list.dump'))
obj_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in obj_pose_list]]

### Pixels

In [5]:
images = pickle.load(open('image_list.dump'))
camera_matrix, dist_coeffs = calibrate_lens(images)

In [6]:
intrinsics = [(camera_matrix[0,0] + camera_matrix[1,1]) / 2,
              camera_matrix[0,2], camera_matrix[1,2]] + \
              dist_coeffs.tolist()

In [7]:
corners = [corners for corners in 
           [find_corners(image, pattern_size)[1].reshape(-1,2) 
            for image in images]]

### Initital estimate

In [36]:
n_lines = len(rob_motors)
lines_a = []
lines_b = []
for i in range(n_lines):
    for j in range(i+1,n_lines):
        lines_a.append(((rob_motors[j].rev() * rob_motors[i]).log() * 0.5).unit())
        lines_b.append(((obj_motors[j].rev() * obj_motors[i]).log() * 0.5).unit())
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(b,a)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
mes.max_num_iterations = 25
mes.function_tolerance = 1e-12
estimated_hand_eye_motor, summary = mes.solve()
print(estimated_hand_eye_motor)
bivectors = [np.array((r * estimated_hand_eye_motor * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
estimated_camera_robot_motor = vsr.Dll(*np.average(bivectors,0)).exp()
print(estimated_camera_robot_motor)
print(summary['full_report'])

Mot: [ 0.004 -0.0055 -0.68 -0.74 -0.044 -0.047 0.053 -0.0072 ]
Mot: [ -0.66 -0.26 -0.26 -0.66 -0.3 -0.66 -0.15 -0.1 ]

Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                           210                      210
Residual                                 1260                     1260

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear so

In [38]:
A = rob_motors[0]
X = estimated_hand_eye_motor
Z = estimated_camera_robot_motor
B = obj_motors[0]
vsr.Vec(*pattern_points[0]).null().spin(A * X)

Pnt: [ -0.2568 -0.2212 0.7287 1 0.3229 ]

In [39]:
vsr.Vec(*pattern_points[0]).null().spin(Z * B)

Pnt: [ -0.2609 -0.2065 0.7425 1 0.331 ]

### Hand Eye Calibration

In [40]:
print(estimated_hand_eye_motor)
print(estimated_camera_robot_motor)
print(np.array(intrinsics))

Mot: [ 0.004 -0.0055 -0.68 -0.74 -0.044 -0.047 0.053 -0.0072 ]
Mot: [ -0.66 -0.26 -0.26 -0.66 -0.3 -0.66 -0.15 -0.1 ]
[ 1329.5362   527.8128   368.6073    -0.2983     0.9377    -0.0016
    -0.0005    -1.9702]


In [41]:
print(vsr.Drv(1,0,0).spin(estimated_hand_eye_motor))
print(vsr.Drv(0,1,0).spin(estimated_hand_eye_motor))
print(vsr.Drv(0,0,1).spin(estimated_hand_eye_motor))
print(vsr.Vec(0,0,0).null().spin(estimated_hand_eye_motor))

Drv: [ 0.083 -1 0.014 ]
Drv: [ -1 -0.084 -0.0015 ]
Drv: [ 0.0027 -0.013 -1 ]
Pnt: [ 0.06106 0.08873 0.128 1 0.01399 ]


In [42]:
hec = HandEyeCalibrator(estimated_hand_eye_motor, estimated_camera_robot_motor, intrinsics)
for a, b in zip(rob_motors, obj_motors):
    for p in pattern_points:
        hec.add_residual_block_2(a, b, vsr.Vec(*p).null())
hec.linear_solver_type = "DENSE_QR"
hec.max_num_iterations = 100
hec.num_linear_solve_threads = 1000
hec.num_threads = 1000
hec.set_parameterization("BIVECTOR_GENERATOR")
he, cr, intr, summ = hec.solve()
print(he)
print(cr)
print(intr)
print(summ['full_report'])

Mot: [ 0.0016 -0.0064 -0.68 -0.73 -0.054 -0.057 0.05 -0.016 ]
Mot: [ -0.66 -0.26 -0.26 -0.65 -0.31 -0.66 -0.17 -0.11 ]
[1329.5361737671249, 527.8127667721765, 368.60726444265345, -0.2982520378865284, 0.9377411649999892, -0.0015767729018760857, -0.0005171784934892454, -1.97016543830641]

Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            2                        2
Parameters                                 16                       16
Effective parameters                       12                       12
Residual blocks                          1260                     1260
Residual                                 3780                     3780

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given        

In [28]:

# hec = HandEyeCalibrator(estimated_hand_eye_motor, estimated_camera_robot_motor, intrinsics)

# k = 0
# for i in range(len(rob_motors)):
#     for j in range(len(pattern_points)):
#         k += 1
#         hec.add_residual_block(rob_motors[i], 
#                                vsr.Vec(*pattern_points[j]).null(), 
#                                vsr.Vec(corners[i][j][0],corners[i][j][1],0.0))
        
# hec.linear_solver_type = "DENSE_QR"
# hec.max_num_iterations = 50
# hec.num_linear_solve_threads = 1000
# hec.num_threads = 1000
# hec.set_parameterization("BIVECTOR_GENERATOR")
# he, cr, intr, summ = hec.solve()
# print(he)
# print(cr)
# print(intr)
# print(summ['full_report'])

In [43]:
print(vsr.Drv(1,0,0).spin(he))
print(vsr.Drv(0,1,0).spin(he))
print(vsr.Drv(0,0,1).spin(he))
print(vsr.Vec(0,0,0).null().spin(he))

Drv: [ 0.071 -1 0.012 ]
Drv: [ -1 -0.071 -0.0065 ]
Drv: [ 0.0073 -0.011 -1 ]
Pnt: [ 0.04445 0.09669 0.157 1 0.01799 ]


In [30]:
print(vsr.Drv(1,0,0).spin(cr))
print(vsr.Drv(0,1,0).spin(cr))
print(vsr.Drv(0,0,1).spin(cr))
print(vsr.Vec(0,0,0).null().spin(cr))

Drv: [ 0.73 -0.68 -0.011 ]
Drv: [ 0.0027 0.018 -1 ]
Drv: [ 0.68 0.73 0.015 ]
Pnt: [ -0.9877 -0.8758 0.7416 1 1.146 ]
